In [33]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy
from multiprocessing import Pool, Process, Value
import time
from threading import Thread

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep=',', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,0
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,0
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks',
 u'cusELEMENTAL',
 u'cusGOLEM'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks',
            'IS_UNDEAD',
            'cusELEMENTAL',
            'cusGOLEM'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,
1,Halabardnik,115,115,10,5,6,5,2,3,0,
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [7]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.name = unit.name
        
        self.hp = unit.hp
        self.hp_left = self.hp
        
        self.speed = unit.speed
        self.attack = unit.attack
        self.defense = unit.defense
        self.dmg_min = unit.dmg_min
        self.dmg_max = unit.dmg_max
        self.shots = unit.shots
        self.attributes = unit.attributes      
        self.fight_value = unit.fight_value
        self.ai_value = unit.ai_value
        
        self.cap = self.count
        self.rebirth_available = self.name == 'Feniks' or False
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.hp)
            self.count -= num_killed + 1
            self.hp_left = self.hp - rem
            self.count = max(self.count, 0)
        if self.name == 'Feniks' and not self.is_alive() and self.rebirth_available:
            self.rebirth()
            
            
    def __calc_base_damage(self, other):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        defense = other.defense
        if self.name == 'Behemot':
            defense = int(.6 * defense)
        elif self.name == u'Starożytny behemot':
            defense = int(.2 * defense)
        
        att_to_def = self.attack - defense
        base_dmg_reduction = 0.
        base_dmg_bonus = 0.
        if att_to_def > 0:
            base_dmg_bonus = min(.05 * att_to_def, 3.)
        else:
            base_dmg_reduction = min(.025 * -att_to_def, .7)
            
        return base_dmg, base_dmg_bonus, base_dmg_reduction
    
            
    def attack_melee(self, other, dmg_bonus=0., melee_penalty=False):
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        if self.name == 'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if melee_penalty:
            damage /= 2.
        
        damage = int(damage)
        other.take_dmg(damage)
        
        if self.name == 'Wampirzy lord' and not other.is_nonliving():
            self.drain_life(damage)
        elif self.name == 'Wielka gorgona' and not other.is_nonliving():
            self.death_stare(other)
        
        
    def attack_range(self, other, dmg_bonus=0., range_penalty=False):
        assert self.is_shooter() and self.shots > 0
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if range_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        self.shots -= 1
        
    def drain_life(self, damage_dealt):
        assert self.name == 'Wampirzy lord'
        if self.hp_left + damage_dealt <= self.hp:
            self.hp_left += damage_dealt
        else:
            damage_dealt -= self.hp - self.hp_left
            self.hp_left = self.hp
            res, rem = divmod(damage_dealt, self.hp)
            self.count = min(self.cap, self.count + res)
            if self.count < self.cap and rem:
                self.count += 1
                self.hp_left = rem
                
    def death_stare(self, other):
        assert self.name == 'Wielka gorgona' and not other.is_nonliving()
        to_death_stare = min(np.random.binomial(n=self.count, p=.1), (self.count + 9) / 10)
        other.count = max(0, other.count - to_death_stare)
        
    def rebirth(self):
        assert self.name == 'Feniks' and not self.is_alive() and self.rebirth_available
        certain, rem = divmod(self.count, 5)
        to_rebirth = certain + (np.random.rand() < .2*rem)
        self.count = to_rebirth
        self.rebirth_available = False
        
    def is_alive(self):
        return self.count > 0
        
    def is_shooter(self):
        return 'SHOOTING_ARMY' in self.attributes
    
    def strikes_twice(self):
        return 'const_two_attacks' in self.attributes and not self.is_shooter()
    
    def shoots_twice(self):
        return 'const_two_attacks' in self.attributes and self.is_shooter()
    
    def is_big(self):
        return 'DOUBLE_WIDE' in self.attributes
    
    def melee_penalty(self):
        return self.is_shooter() and not 'const_no_melee_penalty' in self.attributes
    
    def no_retaliation(self):
        return 'const_free_attack' in self.attributes
    
    def range_penalty(self):
        return self.is_shooter() and self.name != 'Strzelec'
    
    def is_elemental(self):
        return 'cusELEMENTAL' in self.attributes
    
    def is_undead(self):
        return 'IS_UNDEAD' in self.attributes
    
    def is_golem(self):
        return 'cusGOLEM' in self.attributes
    
    def is_nonliving(self):
        return self.is_elemental() or self.is_undead() or self.is_golem()

In [8]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [9]:
def fight(stackA, stackB, num_iter):    
    wins = {stackA.name : [0,0],
            stackB.name : [0,0]}
    
    def units_order(s1, s2):
        temp = sorted([s1,s2], key=lambda x: x.speed, reverse=True)
        if s1.speed == s2.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        current.attack_melee(other, melee_penalty=current.melee_penalty())
        
        if other.is_alive() and not current.no_retaliation():
            other.attack_melee(current, melee_penalty=other.melee_penalty())
            
        if current.is_alive() and current.strikes_twice():
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, apply_penalty):
        penalty = current.range_penalty() if apply_penalty else False
        current.attack_range(other, range_penalty=penalty)
        if current.shoots_twice() and current.shots > 0:
            current.attack_range(other, range_penalty=penalty)
        return other, current
    
    def fight_to_death(current, other):
        while current.is_alive() and other.is_alive():
            current, other = melee_hit(current, other)
        return current, other
    
    def walker_vs_shooter(walker, shooter):            
        to_walk = starting_dist - 1
        first_move = to_walk % walker.speed
        if first_move == 0:
            first_move = walker.speed
        avoid_by_move = to_walk - first_move > 10
        avoid_by_wait = False
            
        if walker.speed < shooter.speed:
            num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
        elif walker.speed > shooter.speed:
            num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
            avoid_by_wait = True
        else:
            if np.random.rand() < .5:
                num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
            else:
                num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
                    
        num_full_shots = max(0, num_shots - avoid_by_move - avoid_by_wait)
        num_half_shots = num_shots - num_full_shots
            
        for j in xrange(num_half_shots):
            range_hit(shooter, walker, apply_penalty=True)
        for j in xrange(num_full_shots):
            range_hit(shooter, walker, apply_penalty=False)
            
        current, other = walker, shooter
        return fight_to_death(current, other)
    
    starting_dist = 14
    if stackA.is_big():
        starting_dist -= 1
    if stackB.is_big():
        starting_dist -= 1
        

    for it in xrange(num_iter):
        current, other = units_order(copy(stackA), copy(stackB))
        
        if not current.is_shooter() and not other.is_shooter():
            current, other = fight_to_death(current, other)
                
        elif current.is_shooter() and other.is_shooter():
            full_round = True
            while current.is_alive() and ((current.shots > 0 and other.shots > 0) or not full_round):
                current, other = range_hit(current, other, apply_penalty=True)
                full_round = not full_round
            if current.is_alive():
                if current.shots == 0 and other.shots == 0:
                    current, other = units_order(current, other)
                    current, other = fight_to_death(current, other)
                else:
                    shooter = current if current.shots > 0 else other
                    walker = current if current.shots == 0 else other
                    current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
        else:           
            shooter = current if current.is_shooter() else other
            walker = current if not current.is_shooter() else other
            current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
            
        winner = current if current.is_alive() else other
        wins[winner.name][0] += 1
        wins[winner.name][1] += winner.count
        
    for s in [stackA.name, stackB.name]:
        if wins[s][0]:
            wins[s][1] /= float(wins[s][0])
    
    return wins

In [13]:
def find_balance(nameA, nameB, num_iter):
    
    def balanced(result):
        return (num_iter / 2) * .95 < result[A.name][0] < (num_iter / 2) * 1.05
    
    unitA = make_unit(nameA)
    unitB = make_unit(nameB)
    A = stack(unitA, unitB.ai_value * 10)
    B = stack(unitB, unitA.ai_value * 10)
    
    res = fight(A, B, num_iter)
    
    if balanced(res):
        return A.count, B.count
    
    last_loser = A if res[A.name][0] < res[B.name][0] else B
    inc = last_loser.count / 10
    enough = False
    
    while not enough:
        low = last_loser.count
        last_loser.count += inc
        res = fight(A, B, num_iter)
        loser = A if res[A.name][0] < res[B.name][0] else B
        enough = last_loser is not loser
        if not enough:
            last_loser = loser
        
    if balanced(res):
        return A.count, B.count
    
    high = last_loser.count
    
    # here binsearch from (low, high) for balanced result
    X = last_loser
    
    while True:
        middle = low + (high - low) / 2
        X.count = middle
        res = fight(A, B, num_iter)
        if balanced(res) or np.abs(high - low) <= 1:
            return A.count, B.count
        loser = A if res[A.name][0] < res[B.name][0] else B
        if X is loser:
            low = middle + 1
        else:
            high = middle - 1            

In [14]:
t0 = time.time()
print find_balance(u'Anioł', u'Archanioł', 500)
time.time() - t0

(87760, 54005)


4.834399938583374

In [64]:
t0 = time.time()
A = stack(make_unit(u'Gryf'), 10)
B = stack(make_unit(u'Pikinier'), 30)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Gryf': [27, 2.074074074074074], u'Pikinier': [973, 10.768756423432682]}


0.41163086891174316

In [16]:
castle_units = data.values[:14]
all_units = data.values[:-5]
castle_scores = np.zeros((14, 141))

t0 = time.time()
x_id = 0
i = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        if x[0] == u[0]:
            castle_scores[x_id, u_id] = 1.
        else:
            cnt_x, cnt_u = find_balance(x[0], u[0], 500)
            castle_scores[x_id, u_id] = cnt_u / float(cnt_x)
            print x[0], u[0], castle_scores[x_id, u_id]
            i += 1
            if not i % 20:
                print 'Done {} pairs in {:.2f}s'.format(i, time.time()-t0)
        u_id += 1
    x_id += 1

Pikinier Halabardnik 0.842608695652
Pikinier Łucznik 0.634920634921
Pikinier Kusznik 0.38240917782
Pikinier Gryf 0.365811965812
Pikinier Gryf królewski 0.352678571429
Pikinier Zbrojny 0.215730337079
Pikinier Krzyżowiec 0.159353741497
Pikinier Mnich 0.116295973252
Pikinier Kapłan 0.098643649815
Pikinier Kawalerzysta 0.0680883864337
Pikinier Czempion 0.0618571428571
Pikinier Anioł 0.0255230125523
Pikinier Archanioł 0.0155423883318
Pikinier Centaur 0.994
Pikinier Centaur bojowy 0.869565217391
Pikinier Krasnolud 0.549828178694
Pikinier Krasnoludzki wojownik 0.521531100478
Pikinier Leśny elf 0.330715171558
Pikinier Wysoki elf 0.212539851222
Pikinier Pegaz 0.260617760618
Done 20 pairs in 34.53s
Pikinier Srebrny pegaz 0.253759398496
Pikinier Drzewiec 0.14479638009
Pikinier Ent 0.130386052304
Pikinier Jednorożec 0.0725913621262
Pikinier Jednorożec bitewny 0.0660098522167
Pikinier Zielony smok 0.0303981937603
Pikinier Złoty smok 0.0186230117265
Pikinier Gremlin 1.92045454545
Pikinier Większy gr

In [29]:
np.savez('castle_scores.npz', castle_scores)

In [55]:
a = np.load('castle_scores.npz')['arr_0']
cs = pd.DataFrame.from_records(a.T, columns=data.Singular.values[:14])
cs.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
cs

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,1.000000,1.195426,1.575000,2.622500,2.733645,2.835443,4.625780,6.288770,8.598750,10.107500,14.686792,16.166282,39.180328,64.340176
1,Halabardnik,0.842609,1.000000,1.057047,2.240870,2.394270,2.479247,3.916522,5.480000,6.114783,8.681739,12.811060,14.075067,33.981043,54.306931
2,Łucznik,0.634921,0.946032,1.000000,1.560317,2.994444,3.091787,3.819048,5.350000,4.846825,5.091650,15.684921,17.030159,40.641270,77.485714
3,Kusznik,0.382409,0.447297,0.640569,1.000000,1.907609,2.708152,1.871320,2.778828,2.762500,3.692762,13.583696,14.676630,34.778261,77.504891
4,Gryf,0.365812,0.417664,0.334218,0.524217,1.000000,1.055608,1.655556,2.198291,1.532479,2.911396,5.494071,6.052422,14.611681,23.192389
5,Gryf królewski,0.352679,0.404241,0.323437,0.369775,0.949777,1.000000,1.598214,2.013616,1.464732,2.047991,5.110714,5.712500,14.143750,22.282589
6,Zbrojny,0.215730,0.255329,0.261465,0.535730,0.604027,0.624477,1.000000,1.366517,1.634831,2.024719,3.234170,3.562341,8.668394,13.721076
7,Krzyżowiec,0.159354,0.183004,0.186750,0.344218,0.442177,0.495575,0.732872,1.000000,0.845748,1.574320,2.615240,2.893758,7.044211,10.880238
8,Mnich,0.116296,0.148196,0.206829,0.362276,0.652538,0.682719,0.655376,1.181435,1.000000,1.191611,3.580497,3.882418,9.185578,17.576607
9,Kapłan,0.098644,0.115184,0.196400,0.270800,0.343478,0.488923,0.493896,0.635195,0.838667,1.000000,2.162222,2.828133,6.764933,12.204000


In [57]:
cs.to_csv('castle_scores.csv', encoding='utf-8')

In [60]:
castle_scores_ai_value_based = np.zeros((14, 141))

x_id = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        x_aiv = data['AI Value'].values[x_id]
        u_aiv = data['AI Value'].values[u_id]
        castle_scores_ai_value_based[x_id, u_id] = x_aiv / float(u_aiv)
        u_id += 1
    x_id += 1

In [61]:
np.savez('castle_scores_ai_value_based.npz', castle_scores_ai_value_based)

In [62]:
b = np.load('castle_scores_ai_value_based.npz')['arr_0']
csaiv = pd.DataFrame.from_records(b.T, columns=data.Singular.values[:14])
csaiv.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
csaiv

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,1.000000,1.437500,1.575000,2.300000,4.387500,5.600000,5.562500,7.350000,6.062500,9.375000,24.325000,26.250000,62.737500,109.700000
1,Halabardnik,0.695652,1.000000,1.095652,1.600000,3.052174,3.895652,3.869565,5.113043,4.217391,6.521739,16.921739,18.260870,43.643478,76.313043
2,Łucznik,0.634921,0.912698,1.000000,1.460317,2.785714,3.555556,3.531746,4.666667,3.849206,5.952381,15.444444,16.666667,39.833333,69.650794
3,Kusznik,0.434783,0.625000,0.684783,1.000000,1.907609,2.434783,2.418478,3.195652,2.635870,4.076087,10.576087,11.413043,27.277174,47.695652
4,Gryf,0.227920,0.327635,0.358974,0.524217,1.000000,1.276353,1.267806,1.675214,1.381766,2.136752,5.544160,5.982906,14.299145,25.002849
5,Gryf królewski,0.178571,0.256696,0.281250,0.410714,0.783482,1.000000,0.993304,1.312500,1.082589,1.674107,4.343750,4.687500,11.203125,19.589286
6,Zbrojny,0.179775,0.258427,0.283146,0.413483,0.788764,1.006742,1.000000,1.321348,1.089888,1.685393,4.373034,4.719101,11.278652,19.721348
7,Krzyżowiec,0.136054,0.195578,0.214286,0.312925,0.596939,0.761905,0.756803,1.000000,0.824830,1.275510,3.309524,3.571429,8.535714,14.925170
8,Mnich,0.164948,0.237113,0.259794,0.379381,0.723711,0.923711,0.917526,1.212371,1.000000,1.546392,4.012371,4.329897,10.348454,18.094845
9,Kapłan,0.106667,0.153333,0.168000,0.245333,0.468000,0.597333,0.593333,0.784000,0.646667,1.000000,2.594667,2.800000,6.692000,11.701333


In [65]:
csaiv.to_csv('castle_scores_ai_value_based.csv', encoding='utf-8')

In [71]:
comparison = cs.values[:,1:] / csaiv.values[:,1:]
comparison[np.abs(comparison-1) < .2] = 1.
comparison[comparison == 1.] = ''
comp_df = pd.DataFrame.from_records(comparison, columns=data.Singular.values[:14])
comp_df.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))

comp_df

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,,,,,0.623053,0.506329,,,1.41835,,0.603774,0.615858,0.624512,0.58651
1,Halabardnik,1.21125,,,1.40054,0.784447,0.636414,,,1.4499,1.3312,0.757077,0.770777,0.778605,0.711634
2,Łucznik,,,,,,,,,1.25918,,,,,
3,Kusznik,,0.715675,,,,,0.773759,,,,1.28438,1.28595,1.275,1.62499
4,Gryf,1.605,1.27478,,,,,1.30584,1.31224,,1.36253,,,,
5,Gryf królewski,1.975,1.57478,,,1.21225,,1.60899,1.53418,1.35299,1.22333,,1.21867,1.26248,
6,Zbrojny,,,,1.29565,0.765789,0.620296,,,1.5,1.20133,0.739571,0.754877,0.768566,0.695747
7,Krzyżowiec,,,,,0.740741,0.650442,,,,1.23427,0.790216,,,0.728986
8,Mnich,0.705044,0.625,0.796126,,,0.739104,0.714286,,,0.770575,,,,
9,Kapłan,,0.751202,,,0.733927,,,,1.29691,,,,,


In [72]:
comp_df.to_csv('castle_comparison.csv', encoding='utf-8')